In [0]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import os 

In [0]:
torch.__version__

'1.5.0+cu101'

In [0]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls

drive  sample_data


In [0]:
#with open('/gdrive/My Drive/foo.txt', 'w') as f:
 # f.write('Hello Google Drive!')
#!cat '/gdrive/My Drive/foo.txt'

In [0]:
#%cd drive/My Drive/AlphaPose/AlphaPose-pytorch
#input_path = "/content/drive/My Drive/AlphaPose/AlphaPose-pytorch/fi_data"


In [0]:
#!python3 datalabeler_111.py

In [0]:
%cd drive/My Drive/Resnet_Beta-Version
!ls

/content/drive/My Drive/Resnet_Beta-Version
 acc
'alpha-Resnet(Raw+Face+Hand).ipynb'
'alpha-Resnet(Raw+Face).ipynb'
'alpha-Resnet(Raw+Hand).ipynb'
'alpha-Resnet(Raw).ipynb'
'Arrange Dataset.ipynb'
'beta-Resnet(Raw+Face+Hand).ipynb'
'beta-Resnet(Raw+Face+Hand+Pose)-Pose as Depth.ipynb'
'beta-Resnet(Raw+Face).ipynb'
'beta-Resnet(Raw+Hand).ipynb'
'beta-Resnet(Raw).ipynb'
 dataset
 dump
 epoch_65.pth
 extract_faceandhand_jorehgaye.ipynb
 fallingDemo.ipynb
 fallingDemo_modified_2.py
 fallingDemo_modified.py
 fallingDemo.py
'gamma-Resnet(Raw+Face+Hand).ipynb'
'gamma-Resnet(Raw+Face+Hand+Pose)-Pose as Depth.ipynb'
'gamma-Resnet(Raw+Face).ipynb'
'gamma-Resnet(Raw+Hand).ipynb'
'gamma-Resnet(Raw).ipynb'
 loss
 out.jpg
 putTXT_rightplace.ipynb
'Testbeta-Resnet(Raw+Face+Hand).ipynb'
'Testbeta-Resnet(Raw+Face+Hand+Pose)-Pose as Depth.ipynb'
'Testbeta-Resnet(Raw+Face).ipynb'
'Testbeta-Resnet(Raw+Hand).ipynb'
'Testbeta-Resnet(Raw).ipynb'
 TestingModels_first.ipynb
 TestingModels_second.ipynb
 Testing

In [0]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        #transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}

In [0]:

class MyImageFolder(datasets.ImageFolder):
    
    def __init__(self, root, transform=None):
        super(MyImageFolder,self).__init__(root)

        self.transform = transform
        




    def __getitem__(self, index):

        #imgs = self.imgs[index]
        #print(imgs)

        path, target = self.samples[index]
        #sample = self.loader(path)
        #print("Sample: ",sample)
        

       
        img_path = path
       
        image = Image.open(img_path)
        img_w , img_h = image.size

    
        filename = os.path.basename(img_path)
       
        #Croping images to feeed 
        #if (os.path.isfile(filename.split('.')[0] + '-bbox.txt')):
        cpath  = img_path.replace(filename, filename.split('.')[0] + '-bbox.txt')
        image  = self.crop_text(cpath,image)
        #else:
        #  print("No file "+ filename)
    

        
        # Transforms
        if self.transform is not None:
            
            image = self.transform(image)
        
        return image, target


    def crop_text(self,path,image):
        try:
              fr = open(path)
              # print ("fr",cat+"/yolo/" + imagePath.split("/")[-1].split('.')[0]+".txt")

              line = fr.readline().split(" ")

              width, height = image.size 
        
              # Setting the points ffloor cropped image
              b_w = float(line[3]) * width
              b_h = float(line[4]) * height

              t_left = (float(line[1]) * width) - (b_w/2)
              t_right = (float(line[2]) * height) - (b_h/2)

              b_left = (float(line[1]) * width) + (b_w/2)
              b_right = (float(line[2]) * height) + (b_h/2)

              # print("w",width," h",height)

              # print (t_left)
              # print (t_right)
              # print (b_left)
              # print (b_right)

              image = image.crop((t_left, t_right, b_left, b_right))
              
        except FileNotFoundError:
              print("FileNotFoundError: " + path)
              #pass
        return image 
   
"""
Test Code


path, target = self.samples[index]
sample = self.loader(path)
if self.transform is not None:
  sample = self.transform(sample)
if self.target_transform is not None:
  target = self.target_transform(target)
# this is where you load your resnet data
resnet_path = os.path.join(os.path.splitext(path)[0], '.pth')  # replace image extension with .pth
resnet = torch.load(resnet_path)  # load the stored features
return sample, resnet, target
"""
#dataset = MyImageFolder("falling_dataset/val/")
#re, labels = dataset[1:3]
#print(re)
#print(labels)
#re[2].save('pred_face.jpeg')
#re[3].save('pred_hand.jpeg')

"\nTest Code\n\n\npath, target = self.samples[index]\nsample = self.loader(path)\nif self.transform is not None:\n  sample = self.transform(sample)\nif self.target_transform is not None:\n  target = self.target_transform(target)\n# this is where you load your resnet data\nresnet_path = os.path.join(os.path.splitext(path)[0], '.pth')  # replace image extension with .pth\nresnet = torch.load(resnet_path)  # load the stored features\nreturn sample, resnet, target\n"

In [0]:
#%cd 

In [0]:

input_path = './dataset/alpha/Choke/'

image_datasets = {
    'train': 
    MyImageFolder(input_path+"train" , data_transforms['train']),
    'validation': 
    MyImageFolder(input_path+"val" , data_transforms['validation'])
}

#print(image_datasets['train'][0])

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=64,
                                shuffle=True,
                                num_workers=8),  # for Kaggle
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=32,
                                shuffle=False,
                                num_workers=8)  # for Kaggle
}

In [0]:
#image_datasets['train']=LimitDataset(image_datasets['train'], n=1500)
dataloaders['train'].dataset.classes


['Choking', 'NonChoking']

In [0]:
dataloaders['validation'].dataset.classes

['Choking', 'NonChoking']

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [0]:
model = models.resnet50(pretrained=True).to(device)
    
for param in model.parameters():
    param.requires_grad = False   
    
model.fc = nn.Sequential(
               nn.Linear(2048, 128),
               nn.ReLU(inplace=True),
               nn.Linear(128, 2),
               nn.Sigmoid()).to(device)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/checkpoints/resnet50-19c8e357.pth


In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(),lr=0.001)

In [0]:
def train_model(model, criterion, optimizer, num_epochs=3):
    new_write = True
    for epoch in range(num_epochs):
        
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)
        
        #f.write('Epoch {}/{}'.format(epoch+1, num_epochs))
        

        for phase in ['train','validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            count = 0
            
            with torch.set_grad_enabled(phase == 'train'):
              for inputs, labels in dataloaders[phase]:
                  inputs = inputs.to(device)
                  labels = labels.to(device)

                  outputs = model(inputs)
                  loss = criterion(outputs, labels)

                  if phase == 'train':
                      
                      optimizer.zero_grad()
                      loss.backward()
                      optimizer.step()

                  _, preds = torch.max(outputs, 1)
                  running_loss += loss.item() * inputs.size(0)
                  running_corrects += torch.sum(preds == labels.data)

                  count = count+1                
                  if (count%40==0):
                    print (count)
             

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,epoch_loss,epoch_acc))
            if (os.path.isfile('loss/gamma/Choke/raw.txt')):
              new_write=False  
            else :
              print ("No file")
            f = open('loss/gamma/Choke/raw.txt','a+')
            if  (new_write == True ):
              print ("here")
              f.write('{} loss: {:.4f}, acc: {:.4f}'.format(phase,epoch_loss,epoch_acc))
            else :
              f.write ('\n')
              f.write('{} loss: {:.4f}, acc: {:.4f}'.format(phase,epoch_loss,epoch_acc))
            
            
        torch.save(model, '/content/drive/My Drive/Fyp/gamma/Choke/raw/epoch_%d.pth'%(epoch))
        f.close()


    return model

In [0]:
#count = 1
#torch.save(model, 'weights/pytorch/fallweightsresNet_3_12_2019%d.pth'%(count))

In [0]:
model_trained = train_model(model, criterion, optimizer, num_epochs=100)


Epoch 1/100
----------
FileNotFoundError: ./dataset/alpha/Choke/train/NonChoking/VID20200419005215_220-bbox.txt
FileNotFoundError: ./dataset/alpha/Choke/train/NonChoking/VID_20200415_165905(0)_frame-35-bbox.txt
FileNotFoundError: ./dataset/alpha/Choke/train/NonChoking/VID_20200415_165905(0)_frame-30-bbox.txt
FileNotFoundError: ./dataset/alpha/Choke/train/NonChoking/VID_20200415_171633(0)_frame-910-bbox.txt
40
FileNotFoundError: ./dataset/alpha/Choke/train/NonChoking/VID_20200415_165905(0)_frame-40-bbox.txt
FileNotFoundError: ./dataset/alpha/Choke/train/NonChoking/VID_20200415_171633(0)_frame-915-bbox.txt
train loss: 0.6063, acc: 0.6945
No file
here
FileNotFoundError: ./dataset/alpha/Choke/val/NonChoking/VID_20200415_171633(0)_frame-925-bbox.txt
validation loss: 0.5589, acc: 0.7446
Epoch 2/100
----------
FileNotFoundError: ./dataset/alpha/Choke/train/NonChoking/VID20200419005215_220-bbox.txt
FileNotFoundError: ./dataset/alpha/Choke/train/NonChoking/VID_20200415_165905(0)_frame-30-bbox.t